In [1]:
# send message to Confluent-Kafka Topic via python
bootstrap.servers=pkc-921jm.us-east-2.aws.confluent.cloud:9092
security.protocol=SASL_SSL
sasl.mechanisms=PLAIN
sasl.username=54ETNHYPAUELF65K
sasl.password=cflt1FYSftSYI7sJCXe4ogLdMx73R7OUVKmqHweIQeVBlu+m9WqeC63lCJdSFoFw

# Best practice for higher availability in librdkafka clients prior to 1.7
session.timeout.ms=45000

client.id=ccloud-python-client-2eecfd2f-712e-4fcf-b3e5-b26c610ccfc2


SyntaxError: invalid imaginary literal (ipython-input-3530332403.py, line 2)

In [3]:
!pip install confluent-kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 7.1 MB/s eta 0:00:00


In [7]:
import pandas as pd
import json

csv_file = '/content/Customer_data/first_100_customers.csv'

df =  pd.read_csv(csv_file)
df.head()

,customer_id,name,city,state,country,registration_date,is_active
0,0,Customer_0,Pune,Maharashtra,India,2023-06-29,False
1,1,Customer_1,Bangalore,Tamil Nadu,India,2023-12-07,True
2,2,Customer_2,Hyderabad,Gujarat,India,2023-10-27,True
3,3,Customer_3,Bangalore,Karnataka,India,2023-10-17,False
4,4,Customer_4,Ahmedabad,Karnataka,India,2023-03-14,False


# New Section

In [8]:
json_records =  df.to_dict(orient ='records')
json_file = 'customers.json'
with open(json_file ,'w') as file:
  json.dump(json_records,file,indent =4)

In [9]:
from confluent_kafka import Producer
import json
import time


conf = {
"bootstrap.servers": "pkc-921jm.us-east-2.aws.confluent.cloud:9092",
"security.protocol":"SASL_SSL",
"sasl.mechanisms":"PLAIN",
"sasl.username":"54ETNHYPAUELF65K",
 "sasl.password" :"cflt1FYSftSYI7sJCXe4ogLdMx73R7OUVKmqHweIQeVBlu+m9WqeC63lCJdSFoFw",
'session.timeout.ms':45000,

'client.id': 'ccloud-python-client-2eecfd2f-712e-4fcf-b3e5-b26c610ccfc2'
  }


In [10]:
producer = Producer(conf)

In [11]:
topic = 'ecommerce'

with open ('customers.json' , 'r') as file:
  customers_data = json.load(file)

  value = customers_data[0]
  key = value['customer_id']

  print (key ,value)

0 {'customer_id': 0, 'name': 'Customer_0', 'city': 'Pune', 'state': 'Maharashtra', 'country': 'India', 'registration_date': '2023-06-29', 'is_active': False}


In [12]:
type(str(value).encode('utf-8'))

bytes

In [ ]:
producer.produce(topic,key =str(key).encode('utf-8'),value =str(value).encode('utf-8'))

#Send Multiple Message to our kafaka cluster

In [14]:
def delivery_status(err,msg):
  if(err):
    print(f"Message deliver failed: {err}")
  else:
      print(f"message delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}")

for record in customers_data:
  try:
    message_value =json.dumps(record)
    message_key = str(record['customer_id']).encode('utf-8')
    producer.produce(topic,key = message_key , value =message_value ,callback = delivery_status)
    producer.poll(1)

  except Exception as e:
    print(f"Error sending messages: {e}")

producer.flush()


message delivered to ecommerce [2] at offset 68
message delivered to ecommerce [2] at offset 69
message delivered to ecommerce [1] at offset 111
message delivered to ecommerce [1] at offset 112
message delivered to ecommerce [1] at offset 113
message delivered to ecommerce [1] at offset 114
message delivered to ecommerce [1] at offset 115
message delivered to ecommerce [0] at offset 126
message delivered to ecommerce [2] at offset 70
message delivered to ecommerce [0] at offset 127
message delivered to ecommerce [0] at offset 128
message delivered to ecommerce [0] at offset 129
message delivered to ecommerce [0] at offset 130
message delivered to ecommerce [2] at offset 71
message delivered to ecommerce [0] at offset 131
message delivered to ecommerce [1] at offset 116
message delivered to ecommerce [0] at offset 132
message delivered to ecommerce [2] at offset 72
message delivered to ecommerce [0] at offset 133
message delivered to ecommerce [1] at offset 117
message delivered to ecom

0